In [ ]:
import os
import torch
import pandas as pd
from torch import nn
from torch.utils import data
from d2l import torch as d2l
from sklearn.model_selection import train_test_split

In [ ]:
# 加载数据集
data_file = os.path.join('..', 'data', 'data_salmonbass.xlsx')
data_raw = pd.read_excel(data_file)

In [ ]:
# 数据处理

# 独热编码
# type_mapping = {type: idx for idx, type in enumerate(set(data_raw['type']))}
# data_raw['type'] = data_raw['type'].map(type_mapping)
data_raw = pd.get_dummies(data_raw)
# 特征矩阵和标签
inputs, outputs = data_raw.iloc[:, 0:2], data_raw.iloc[:, 2:4]
features, labels = torch.Tensor(inputs.values), torch.Tensor(outputs.values)

data_raw, inputs, outputs;

In [ ]:
# 网络模型
net = nn.Sequential(nn.Linear(2, 128), nn.ReLU(), nn.Linear(128, 64),
                    nn.ReLU(), nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 16),
                    nn.ReLU(), nn.Linear(16, 2))


# 初始化权重
def init_weight(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)


net.apply(init_weight)


In [ ]:
# 损失函数
loss = nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 优化算法
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [ ]:
def kfolder(features, labels, k_fold=2):
    """k折交叉验证"""
    for _ in range(k_fold):
        train_x, test_x, train_y, test_y = train_test_split(features,
                                                            labels,
                                                            test_size=1 /
                                                            k_fold)
        train_x = torch.Tensor(train_x)
        train_y = torch.Tensor(train_y)
        test_x = torch.Tensor(test_x)
        test_y = torch.Tensor(test_y)
        yield (train_x, train_y), (test_x, test_y)


In [ ]:
def data_iter(data_arrays, batch_size, is_train=True):
    """PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [ ]:
def train(net, train_iter, test_iter, loss, num_epochs, updater):

    animator = d2l.Animator(xlabel='epoch',
                            xlim=[1, num_epochs],
                            ylim=[0.3, 0.9],
                            legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch(net, train_iter, loss, updater)
        test_acc = d2l.evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc, ))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
def train_epoch(net, train_iter, loss, updater):

    # Set the model to training mode
    if isinstance(net, torch.nn.Module):
        net.train()
    # Sum of training loss, sum of training accuracy, no. of examples
    metric = d2l.Accumulator(3)
    for X, y in train_iter:
        # Compute gradients and update parameters
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # Using PyTorch in-built optimizer & loss criterion
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # Using custom built optimizer & loss criterion
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # Return training loss and training accuracy
    return metric[0] / metric[2], metric[1] / metric[2]


In [ ]:
def accuracy(y_hat, y):

    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = d2l.argmax(y_hat, axis=1)
    cmp = d2l.astype(y_hat, y.dtype) == y
    return float(d2l.reduce_sum(d2l.astype(cmp, y.dtype)))

In [ ]:
# 训练（未实现）
k = 10
num_epochs = 10
batch_size = 10
for trainset, testset in kfolder(features, labels, k_fold=k):
    train_iter = data_iter(trainset, batch_size)
    test_iter = data_iter(testset, batch_size)
    train(net, train_iter, test_iter, loss, num_epochs, trainer)